In [1]:
import warnings
import sys
sys.path.append('../../..')
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from models import *
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
train_path = '../../../data/CICIDS2018/train_dataset_treated.parquet'
test_path = '../../../data/CICIDS2018/test_dataset_treated.parquet'

In [2]:
data_train = pd.read_parquet(train_path)
data_test = pd.read_parquet(test_path)

X_train = data_train.drop(columns=['Label']).astype(np.float32)
X_test = data_test.drop(columns=['Label']).astype(np.float32)

y_train = np.asarray(data_train['Label'].astype('category').cat.codes, np.int64)
y_test = np.asarray(data_test['Label'].astype('category').cat.codes, np.int64)

In [3]:
gate = FeedForward(X_train.shape[1], 14, [256, 128])

In [4]:
logger = TensorBoardLogger("logs", name="gate_tensorboard")
csv_logger = pl.loggers.CSVLogger("logs", name="gate_csv")
lr_monitor = LearningRateMonitor(logging_interval='epoch')

train_loader = DataLoader(TensorDataset(torch.tensor(X_train.values, device='cuda'), 
                                torch.tensor(y_train, device='cuda')), batch_size=4096, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.tensor(X_test.values, device='cuda'), 
                                        torch.tensor(y_test, device='cuda')), batch_size=4096)


trainer = pl.Trainer(                
        max_epochs=50,
        logger=[logger, csv_logger],
        callbacks=[lr_monitor],
        accelerator='gpu',
)

trainer.fit(gate, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-10-01 15:48:01.537321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 15:48:01.577209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 15:48:01.588642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]